In [1]:
import numpy as np
import scipy.optimize as optimize

In [2]:
""" Optimization Algorithm """
""" New Matrix """
def newMat(x, Vt, k):
  V_new = np.zeros((Vt.shape), dtype=np.cfloat)
  if k==2:
    V_new[0] = np.cos(x[0])
    V_new[1] = (np.sin(x[0])) * np.exp(1j*x[1])
  elif k==3:
    V_new[0] = np.cos(x[0])
    V_new[1] = (np.sin(x[0])) * (np.cos(x[1])) * np.exp(1j*x[2])
    V_new[2] = (np.sin(x[0])) * (np.sin(x[1])) * np.exp(1j*x[3])
  else:
    V_new[0] = (np.cos(x[0])) * (np.cos(x[1]))
    V_new[1] = (np.cos(x[0])) * (np.sin(x[1])) * np.exp(1j*x[3])
    V_new[2] = (np.sin(x[0])) * (np.cos(x[2])) * np.exp(1j*x[4])
    V_new[3] = (np.sin(x[0])) * (np.sin(x[2])) * np.exp(1j*x[5])
  return V_new


""" Cost Function """
def costFn(x, Ut, Vt, A, k):
    V_new = newMat(x, Vt, k)
    Bp = np.dot(Ut, V_new) 
    loss = np.linalg.norm(A - Bp*np.conjugate(Bp))
    return (loss)


In [3]:
def calcResults(k, k_new):
    final_res = []
    m = k+5
    for n in range(k+1, k+5):
        print ("m = ",m,", n = ",n)
        res = np.zeros((100,3))
        for i in range(100):
            A = np.random.rand(m, n)
            A = A/A.sum(axis=0)         # Optimize column-wise

            #Classic Truncated SVD
            U, L, V = np.linalg.svd(A, full_matrices=False)
            Ut = U[:, :k]
            Vt = V[:k]
            Lt = L[:k]
            At = np.dot(np.dot(Ut,np.diag(Lt)), Vt)
            res[i][0] = (np.linalg.norm(A - At))


            # Complex SVD
            B = np.sqrt(A)
            U, L, V = np.linalg.svd(B, full_matrices=False)
            # Complex SVD with k
            if (k<=4):                                      # Skip when k>4
                Ut = U[:, :k]
                Vt = V[:k]
                Lt = L[:k]
                initial_guess = np.ones((2*n*(k-1),), dtype=np.longdouble)
                V_new = np.zeros(Vt.shape, dtype=np.cfloat)
                for col in range(Vt.shape[1]):
                    result = optimize.minimize(fun=costFn, x0=initial_guess, args=(Ut,Vt[:, col],A[:,col],k),
                                            tol=1e-7, method='Nelder-Mead', options={'maxiter':1e+10})
                    V_new[:,col] = newMat(result.x, Vt[:, col], k)
                Bp = np.dot(Ut, V_new)  
                res[i][1] = (np.linalg.norm(A - np.conjugate(Bp)*Bp))
            else:
                res[i][1] = (np.linalg.norm(A - At))

            # Complex SVD with k_new
            Ut = U[:, :k_new]
            Vt = V[:k_new]
            Lt = L[:k_new]
            initial_guess = np.ones((2*n*(k_new-1),), dtype=np.longdouble)
            V_new = np.zeros(Vt.shape, dtype=np.cfloat)
            for col in range(Vt.shape[1]):
                result = optimize.minimize(fun=costFn, x0=initial_guess, args=(Ut,Vt[:, col],A[:,col],k_new),
                                        tol=1e-7, method='Nelder-Mead', options={'maxiter':1e+10})
                V_new[:,col] = newMat(result.x, Vt[:, col], k_new)
            Bp = np.dot(Ut, V_new)  
            res[i][2] = (np.linalg.norm(A - np.conjugate(Bp)*Bp))


            if i%10==0: print(i, end=' ')
        print('\n')
        final_res.append([m, n, *res.mean(axis=0)])
    return final_res

In [4]:
res = calcResults(k=5, k_new=4)

m =  10 , n =  6
0 10 20 30 40 50 60 70 80 90 

m =  10 , n =  7
0 10 20 30 40 50 60 70 80 90 

m =  10 , n =  8
0 10 20 30 40 50 60 70 80 90 

m =  10 , n =  9
0 10 20 30 40 50 60 70 80 90 



In [5]:
res

[[10, 6, 0.07277964670071375, 0.07277964670071375, 0.11263416005227313],
 [10, 7, 0.11068679376422773, 0.11068679376422773, 0.14166369846508212],
 [10, 8, 0.14439628197819115, 0.14439628197819115, 0.1716759823632197],
 [10, 9, 0.1724780190348054, 0.1724780190348054, 0.19224986759658771]]

In [9]:
combo = []
for k1 in range(2, 12):
    for m in range(k1+1, k1+9):
        for n in range(k1+1, k1+9):
            k_new = (k1*(m+n-1)+2*n)/(2*n+m-1)
            if (k2%1==0):
                if [k1, k2] not in combo: 
                    combo.append([k1, k2])
print(combo)

[[2, 2.0], [5, 4.0], [7, 5.0], [8, 6.0], [9, 7.0], [10, 7.0], [11, 8.0]]
